## Task 1 : sentiment Labeling
#### the installation instructions for setting up the libraries
Install core libraries: 
pip install pandas spacy python-dateutil

Install SpacyTextBlob for sentiment analysis:
pip install spacytextblob


In [2]:
#import the libiaries
import pandas as pd
import re
from datetime import timedelta
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
# read csv file
df = pd.read_csv('testin.csv')

# Clean text function
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text
df['body'] = df['body'].apply(clean_text)

In [3]:
#Load spaCy and add TextBlob component
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("spacytextblob")

# Define a function to classify sentiment using TextBlob
def classify_sentiment(text):
    doc = nlp(text)
     # get the sentiment score
    sentiment_score = doc._.blob.polarity
    # get the sentiment label
    if sentiment_score > 0.4:
        sentiment_label = "Positive"
    elif sentiment_score < 0:
        sentiment_label = "Negative"
    else:
        sentiment_label = "Neutral"
    return sentiment_label

df['sentiment_label'] = df['body'].apply(classify_sentiment)

# Save the updated DataFrame to a new CSV file !
# df = pd.read_excel('testin.csv') (if you need)

# Check the result
df

,Subject,body,date,from,sentiment_label
0,EnronOptions Update!,enronoptions announcement we have updated the ...,5/10/2010,sally.beck@enron.com,Neutral
1,(No Subject),marc unfortunately today is not going to work ...,7/29/2010,eric.bass@enron.com,Negative
2,Phone Screen Interview - Shannon L. Burnham,when wednesday june 06 2001 1000 am1100 am gmt...,7/25/2011,sally.beck@enron.com,Neutral
3,RE: My new work email,we were thinking papasitos we can meet somewhe...,3/25/2010,johnny.palmer@enron.com,Negative
4,Bet,since you never gave me the 20 for the last ti...,5/21/2011,lydia.delgado@enron.com,Negative
...,...,...,...,...,...
2186,Re: Resume,thanks for the resume she has had some good ex...,6/17/2011,johnny.palmer@enron.com,Neutral
2187,"Final Schedule - Wednesday, May 2, 2001 - Jesu...",attached please find the following documents t...,1/20/2011,johnny.palmer@enron.com,Neutral
2188,(No Subject),good to finally hear from judging from your em...,1/2/2011,don.baughman@enron.com,Neutral
2189,League is Set,it looks like we have our 12 teams we will pro...,3/11/2011,rhonda.denton@enron.com,Neutral


## Task 2: Go to Task 2 EDA.ipynb file

## Task 3: Employee Score Calculation

In [4]:
# Prepare Employee Score for ranking
df['month'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m')
df['Label_score'] = df['sentiment_label']
df['Label_score'] = df['Label_score'].str.replace('Negative','-1').replace('Positive','1').replace('Neutral','0')
df['Label_score'] = df['Label_score'].astype(int)  
df #check again

,Subject,body,date,from,sentiment_label,month,Label_score
0,EnronOptions Update!,enronoptions announcement we have updated the ...,5/10/2010,sally.beck@enron.com,Neutral,2010-05,0
1,(No Subject),marc unfortunately today is not going to work ...,7/29/2010,eric.bass@enron.com,Negative,2010-07,-1
2,Phone Screen Interview - Shannon L. Burnham,when wednesday june 06 2001 1000 am1100 am gmt...,7/25/2011,sally.beck@enron.com,Neutral,2011-07,0
3,RE: My new work email,we were thinking papasitos we can meet somewhe...,3/25/2010,johnny.palmer@enron.com,Negative,2010-03,-1
4,Bet,since you never gave me the 20 for the last ti...,5/21/2011,lydia.delgado@enron.com,Negative,2011-05,-1
...,...,...,...,...,...,...,...
2186,Re: Resume,thanks for the resume she has had some good ex...,6/17/2011,johnny.palmer@enron.com,Neutral,2011-06,0
2187,"Final Schedule - Wednesday, May 2, 2001 - Jesu...",attached please find the following documents t...,1/20/2011,johnny.palmer@enron.com,Neutral,2011-01,0
2188,(No Subject),good to finally hear from judging from your em...,1/2/2011,don.baughman@enron.com,Neutral,2011-01,0
2189,League is Set,it looks like we have our 12 teams we will pro...,3/11/2011,rhonda.denton@enron.com,Neutral,2011-03,0


## Task 4 : Employee Ranking

In [5]:
# Calculate monthly scores
monthly_scores = df.groupby(['month', 'from'], dropna=False)['Label_score'].sum().reset_index()

# Get top 3 per month
top_3 = (monthly_scores.sort_values(['month', 'Label_score'], ascending=[True, False])
                     .groupby('month')
                     .head(3)
                     .reset_index(drop=True))

# Format output for ALL months
def format_all_months(df):
    output = []    #Creates an empty list to accumulate the formatted strings.
    current_month = None  #Tracks which month is being processed (starts with no month)
    
    for _, row in df.iterrows():  #Iterates over each row in the DataFrame
        if row['month'] != current_month:
            output.append(f"\n=== {row['month']} ===")
            current_month = row['month']
            rank = 1
        else:
            rank += 1
            
        output.append(f"{rank}. {row['from']}: {row['Label_score']} (Total Score)")
    
    return '\n'.join(output) #Joins all strings in output with newlines and returns the result.

print("COMPLETE MONTHLY TOP 3 RANKINGS")
print("===============================")
print(format_all_months(top_3))

COMPLETE MONTHLY TOP 3 RANKINGS

=== 2010-01 ===
1. eric.bass@enron.com: 1 (Total Score)
2. patti.thompson@enron.com: 1 (Total Score)
3. bobette.riner@ipgdirect.com: 0 (Total Score)

=== 2010-02 ===
1. john.arnold@enron.com: 2 (Total Score)
2. bobette.riner@ipgdirect.com: 0 (Total Score)
3. eric.bass@enron.com: 0 (Total Score)

=== 2010-03 ===
1. bobette.riner@ipgdirect.com: 0 (Total Score)
2. don.baughman@enron.com: 0 (Total Score)
3. kayne.coulter@enron.com: 0 (Total Score)

=== 2010-04 ===
1. don.baughman@enron.com: 1 (Total Score)
2. sally.beck@enron.com: 1 (Total Score)
3. bobette.riner@ipgdirect.com: 0 (Total Score)

=== 2010-05 ===
1. bobette.riner@ipgdirect.com: 0 (Total Score)
2. eric.bass@enron.com: 0 (Total Score)
3. johnny.palmer@enron.com: 0 (Total Score)

=== 2010-06 ===
1. don.baughman@enron.com: 2 (Total Score)
2. patti.thompson@enron.com: 1 (Total Score)
3. eric.bass@enron.com: 0 (Total Score)

=== 2010-07 ===
1. don.baughman@enron.com: 0 (Total Score)
2. john.arnold@e

In [6]:
# Get bad 3 per month
resq = df.groupby(['month','from'], dropna=False).agg({'Label_score':'sum'}).reset_index()
bad_3 = (resq.sort_values(['month','Label_score'], ascending=[True,True]).groupby('month').head(3).reset_index(drop=True))

# Format output for ALL months
def format_all_months(df):
    output = []
    current_month = None
    
    for _, row in df.iterrows():
        if row['month'] != current_month:
            output.append(f"\n=== {row['month']} ===")
            current_month = row['month']
            rank = 1
        else:
            rank += 1
            
        output.append(f"{rank}. {row['from']}: {row['Label_score']} (Total Score)")
    
    return '\n'.join(output)

print("COMPLETE MONTHLY BAD 3 RANKINGS")
print("===============================")
print(format_all_months(bad_3))

COMPLETE MONTHLY BAD 3 RANKINGS

=== 2010-01 ===
1. john.arnold@enron.com: -1 (Total Score)
2. johnny.palmer@enron.com: -1 (Total Score)
3. kayne.coulter@enron.com: -1 (Total Score)

=== 2010-02 ===
1. don.baughman@enron.com: -2 (Total Score)
2. johnny.palmer@enron.com: -2 (Total Score)
3. sally.beck@enron.com: -2 (Total Score)

=== 2010-03 ===
1. john.arnold@enron.com: -2 (Total Score)
2. johnny.palmer@enron.com: -2 (Total Score)
3. eric.bass@enron.com: -1 (Total Score)

=== 2010-04 ===
1. eric.bass@enron.com: -1 (Total Score)
2. johnny.palmer@enron.com: -1 (Total Score)
3. kayne.coulter@enron.com: -1 (Total Score)

=== 2010-05 ===
1. patti.thompson@enron.com: -3 (Total Score)
2. sally.beck@enron.com: -3 (Total Score)
3. john.arnold@enron.com: -2 (Total Score)

=== 2010-06 ===
1. john.arnold@enron.com: -3 (Total Score)
2. bobette.riner@ipgdirect.com: -1 (Total Score)
3. johnny.palmer@enron.com: -1 (Total Score)

=== 2010-07 ===
1. sally.beck@enron.com: -6 (Total Score)
2. bobette.rine

## Task 5 : Flight Risk Identification

In [7]:
# read csv file
pf = pd.read_csv('testin.csv')
pf['date'] = pd.to_datetime(pf['date'])

# Filter only negative emails
negative_emails = pf[pf['sentiment_label'] == 'Negative'].copy()

# Function to check rolling 30-day window for each employee
def has_4_in_30_days(group):
    dates = group['date'].sort_values().reset_index(drop=True)
    for i in range(len(dates)):
        window_end = dates[i] + timedelta(days=30) #For each date (dates[i]), define a 30-day window ending at window_end
        count = ((dates >= dates[i]) & (dates <= window_end)).sum() #Uses boolean masking to count how many dates fall within [dates[i], window_end]
        if count >= 4:
            return True
    return False

# Apply the function to each employee group
result = negative_emails.groupby('from').apply(has_4_in_30_days)

# Get the list of employees who meet the criteria
flagged_employees = result[result].index.tolist()

# Convert employee list to DataFrame and more formating
data = flagged_employees
pf = pd.DataFrame(data, columns=['Negative_employees in Row'])
print(pf)

     Negative_employees in Row
0  bobette.riner@ipgdirect.com
1       don.baughman@enron.com
2          eric.bass@enron.com
3        john.arnold@enron.com
4      johnny.palmer@enron.com
5      kayne.coulter@enron.com
6      lydia.delgado@enron.com
7     patti.thompson@enron.com
8      rhonda.denton@enron.com
9         sally.beck@enron.com


C:\Users\kikil\AppData\Local\Temp\ipykernel_12196\1073489284.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = negative_emails.groupby('from').apply(has_4_in_30_days)


## Task 6: Go to Task 6 Predictive Modeling.ipynb file